## More Advanced Python

There are several advanced Python notions.

### Decorators

**Decorator:** is a function that takes another function as input, adds some functionality to it, and returns it. we can apply decorators using the @decorator_name syntax.

Below is an example of simple decorator with argument (the only input is the function):

In [4]:
# Define a simple decorator
def my_decorator(func): #take a function as input
    def wrapper(): #wrapper is a function that has same functionality as the input function but with added functionality
        print("Something is happening before the function is called.") #added functionality 1
        func()
        print("Something is happening after the function is called.") #added functionality 2
    return wrapper #return the modified function

# Define a function and decorate it
@my_decorator
def say_hello():
    print("Hello!")

# Call the decorated function
say_hello()

Something is happening before the function is called.
Hello!
Something is happening after the function is called.


Decorator with Arguments:

In [ ]:
# we use:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("Something is happening before the function is called.")
        func(*args, **kwargs)
        print("Something is happening before the function is called.")
    return wrapper

@my_decorator
def greet(name):
    print(f"Hello, {name}!")

greet("Yuhao")


# rather than:
def my_decorator(func):
    def wrapper(name):
        print("Something is happening before the function is called.")
        func(name)
        print("Something is happening before the function is called.")
    return wrapper

@my_decorator
def greet(name):
    print(f"Hello, {name}!")

greet("Yuhao")

# this is because the first method allows more flexibility, we can apply the decorator to any function, regardless to how many positional arguments and keyword arguments it have

Something is happening before the function is called.
Hello, Yuhao!
Something is happening before the function is called.


Buid a repeat decorator: repeat input function for specified times

In [ ]:
# why this work
def repeat_decorator(times): #decorator with argument other than function must returns a decorator with function as only input
    def decorator(func): #then this returned decorator is applied to the input function (with pre-defined times argument), output a wrapper
        def wrapper(*args, **kwarg): #the wrapper, which is the modified version of input function, repeats the input function with specified times
            for _ in range(times):
                func(*args, **kwarg)
        return wrapper
    return decorator

@repeat_decorator(times=3)
def greet(name):
    print(f"Hello, {name}!")

greet("yuhao")


Hello, yuhao!
Hello, yuhao!
Hello, yuhao!


When we wrap a function using a decorator, the original function’s metadata (like its name and docstring) can be lost (replaced by wrapper's meta data). we use functools.wraps to preserve it:

With out functools.wraps:

In [11]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        """Wrapper function."""
        print("Calling decorated function...")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def example_function():
    """This is an example function."""
    print("Example function is running!")

print(example_function.__name__)
print(example_function.__doc__)

wrapper
Wrapper function.


With functools.wraps():

In [12]:
import functools

def my_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        """Wrapper function."""
        print("Calling decorated function...")
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def example_function():
    """This is an example function."""
    print("Example function is running!")

print(example_function.__name__)
print(example_function.__doc__)

example_function
This is an example function.


We can also implement decorator in class, so that we do not need to define a wrapper function:

In [13]:
class MyDecorator:
    def __init__(self, func):
        self.func = func #save the input function as an attribute of the decorator

    def __call__(self, *args, **kwargs): #make the decorator class behave like a function and can be called as a function
        print("Before the function call.") #modification 1
        result = self.func(*args, **kwargs)
        print("After the function call.") #modification 2
        return result

@MyDecorator
def say_hello():
    print("Hello!")

say_hello()

Before the function call.
Hello!
After the function call.


Chaining multiple decorators: we can apply multiple decorators to a function

In [15]:
def uppercase(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper()
    return wrapper

def exclaim(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result + "!"
    return wrapper

@exclaim #the orginal function;s output is firstly uppercased
@uppercase #then added with "!"
def greet(name):
    return f"hello, {name}"

print(greet("Boris"))

HELLO, BORIS!


Decorator example: timing a function

In [16]:
import time

def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Execution time: {end_time - start_time:.4f} seconds")
        return result
    return wrapper

@timing_decorator
def slow_function():
    time.sleep(2)
    print("Finished!")


@timing_decorator
def fast_function():
    time.sleep(0.1)
    print("Finished!")

slow_function()
fast_function()

Finished!
Execution time: 2.0053 seconds
Finished!
Execution time: 0.1010 seconds


Decorator example: vectorizing a function using np.vectorize

In [ ]:
import numpy as np

# Use np.vectorize as a decorator
@np.vectorize
def my_function(x): #a function for scalalr input x
    return x ** 2 if x > 0 else 0

# Apply it to a NumPy array
data = np.array([-2, -1, 0, 1, 2])
result = my_function(data)
print(result)

[0 0 0 1 4]


- np.vectorize is useful for extending scalar functions to arrays when we don’t want to manually iterate over elements.
- However, it doesn’t provide true performance benefits like NumPy’s ufuncs (universal functions)

**Numpy Universal Functions (ufuncs):** a function that operates on ndarrays in an element-by-element fashion, supporting array broadcasting, type casting, and several other standard features.

Square-root, add, and trigonometric functions in numpy are example of ufuncs:

In [19]:
import numpy as np

# Element-wise addition
arr1 = np.array([1, 2, 3])
arr2 = np.array([4, 5, 6])

result = np.add(arr1, arr2)
print(result)  # Output: [5 7 9]


arr = np.array([1, 4, 9, 16])
# Square root
sqrt_result = np.sqrt(arr)
print(sqrt_result)  # Output: [1. 2. 3. 4.]

# Trigonometric functions
angles = np.array([0, np.pi / 2, np.pi])
sin_result = np.sin(angles)
print(sin_result)  # Output: [0. 1. 0.]

[5 7 9]
[1. 2. 3. 4.]
[0.0000000e+00 1.0000000e+00 1.2246468e-16]


We can create a custom ufunc from a element-wise Python function as well:

In [20]:
def custom_add(x, y):
    return x + y

# Create ufunc
custom_add_ufunc = np.frompyfunc(custom_add, 2, 1)

result = custom_add_ufunc([1, 2, 3], [4, 5, 6])
print(result)  # Output: [5 7 9]

[5 7 9]


Here is an example of the huge performance benefits of ufuncs:

In [21]:
import numpy as np
import time

# Data
arr = np.arange(1e6)

# Using ufunc
start = time.time()
result_ufunc = np.sqrt(arr)
end = time.time()
print(f"ufunc time: {end - start:.6f} seconds")

# Using np.vectorize
def scalar_sqrt(x):
    return x ** 0.5

vectorized_sqrt = np.vectorize(scalar_sqrt)
start = time.time()
result_vectorized = vectorized_sqrt(arr)
end = time.time()
print(f"np.vectorize time: {end - start:.6f} seconds")

ufunc time: 0.001444 seconds
np.vectorize time: 0.086611 seconds



In this example, the ufunc version is 100 times faster than the vectorized version. The vectorized version actually iterates over the elements of the array, while the ufunc version is implemented in C under the hood and is highly optimized.

### Logging

- Logging is used to track events in a program and helps debug or monitor it without relying on print statements.
- It is better because it categorizes messages by severity levels (INFO, WARNING, ERROR) and can output logs to different destinations (e.g., files).

Basic setup of logging:

In [2]:
import logging

# Configure basic logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Example:
def compute_square(x):
    if x < 0:
        logging.warning("Received a negative number. Returning zero.") #the Waring log level logs when unexpected input is received.
        return None
    result = x ** 0.5
    logging.info(f"Computed square root of {x}: {result}") # the Info logs useful computation results.
    return result

# Call the function
for number in range(-10,10):
    compute_square(number)

# we use logging because it separates debugging messages from main program output (since it does not use printing statement)
# besides, it iss easy to expand (e.g., write to files, add timestamps).

INFO: Computed square root of 0: 0.0
INFO: Computed square root of 1: 1.0
INFO: Computed square root of 2: 1.4142135623730951
INFO: Computed square root of 3: 1.7320508075688772
INFO: Computed square root of 4: 2.0
INFO: Computed square root of 5: 2.23606797749979
INFO: Computed square root of 6: 2.449489742783178
INFO: Computed square root of 7: 2.6457513110645907
INFO: Computed square root of 8: 2.8284271247461903
INFO: Computed square root of 9: 3.0


Activating and deactivating logging:

Sometimes, we may want to turn logging on or off depending on the situation (e.g., during debugging or production). To deactivate logging, set the logging level to logging.CRITICAL. Since this is the highest level, only critical errors will be logged, effectively “silencing” other logs.

In [3]:
# Deactivate logging
logging.getLogger().setLevel(logging.CRITICAL)

for number in range(-10,10):
    compute_square(number)

To Reactivate Logging, we set the logging level back to logging.INFO:

In [4]:
logging.getLogger().setLevel(logging.INFO)

for number in range(-10,10):
    compute_square(number)

INFO: Computed square root of 0: 0.0
INFO: Computed square root of 1: 1.0
INFO: Computed square root of 2: 1.4142135623730951
INFO: Computed square root of 3: 1.7320508075688772
INFO: Computed square root of 4: 2.0
INFO: Computed square root of 5: 2.23606797749979
INFO: Computed square root of 6: 2.449489742783178
INFO: Computed square root of 7: 2.6457513110645907
INFO: Computed square root of 8: 2.8284271247461903
INFO: Computed square root of 9: 3.0


### Global and Local Variables

Variables in python can have global or local scope:
- **Global variables:** are accessible everywhere in the Python script.
- **Local variables:** are only accessible in the function they are defined in.

Variables created in for, while and if statements are global, so available outside the loop (as are the iterators, e.g. i in for i in range(10)).

In [5]:
for i in range(3):
    j=100
print(j)
print(i)

while i<2:
    j=101
    i=5
print(j)

if (True):
    j=102
print(j)

100
2
100
102


Variables created in functions are local, so not available outside the function:

In [7]:
j = 0
def f():
    j=103
f()
print(j)

0


- When global and local variables have the same name python creates two instances.
- The local variable takes precedence (higher piority) in the function and dies at the end of it, the global variable takes precedence outside the function.

Below is an example: we may want to modify the global variable k in the function by adding 105 to it; however, python execute the code by looking from left to right. So it firstly creating a new local variable k, with values from the righ hand side, but in the right hand side, it find that the local variable k has not been assigned a value, so it reported an error.

In [11]:
k=104
def function2():
    k=105
    return 0
function2()
print(k)

k=104
def function3():
    k = k + 105
    return 0
function3()
# print(k)

104


UnboundLocalError: cannot access local variable 'k' where it is not associated with a value

### Deep and Shallow Copies

For shallow copy of some python object, modify value of orginal object might influence the value of the copied object. Reverse is also true.

In [12]:
# Original list
original = [[1, 2, 3], [4, 5, 6]]

# Shallow copy
shallow = original

# Modify the nested object
shallow[0][0] = 99

print("Original:", original)  # Output: [[99, 2, 3], [4, 5, 6]]
print("Shallow:", shallow)    # Output: [[99, 2, 3], [4, 5, 6]]


original[0][0] = 87

print("Original:", original)  # Output: [[87, 2, 3], [4, 5, 6]]
print("Shallow:", shallow)    # Output: [[87, 2, 3], [4, 5, 6]]

Original: [[99, 2, 3], [4, 5, 6]]
Shallow: [[99, 2, 3], [4, 5, 6]]
Original: [[87, 2, 3], [4, 5, 6]]
Shallow: [[87, 2, 3], [4, 5, 6]]


For deep copy, any changes in original will not influence the copy, vice versa.

In [13]:
import copy
# Original list
original = [[1, 2, 3], [4, 5, 6]]

# Deep copy
deep = copy.deepcopy(original)

# Modify the nested object
deep[0][0] = 99

print("Original:", original)  # Output: [[1, 2, 3], [4, 5, 6]]
print("Deep:", deep)    # Output: [[99, 2, 3], [4, 5, 6]]


original[0][0] = 87

print("Original:", original)  # Output: [[87, 2, 3], [4, 5, 6]]
print("Deep:", deep)    # Output: [[99, 2, 3], [4, 5, 6]]

Original: [[1, 2, 3], [4, 5, 6]]
Deep: [[99, 2, 3], [4, 5, 6]]
Original: [[87, 2, 3], [4, 5, 6]]
Deep: [[99, 2, 3], [4, 5, 6]]


We can also create a deep copy using list comprehension (manually, with out copy library), the same applies to dictionaries.

In [ ]:
original = [[1, 2, 3], [4, 5, 6]]

# Deep copy
deep = [inner[:] for inner in original] #list_name[:] means making a new list with same element

# Modify the nested object
deep[0][0] = 99

print("Original:", original)  # Output: [[1, 2, 3], [4, 5, 6]]
print("Deep:", deep)    # Output: [[99, 2, 3], [4, 5, 6]]


original[0][0] = 87

print("Original:", original)  # Output: [[87, 2, 3], [4, 5, 6]]
print("Deep:", deep)    # Output: [[99, 2, 3], [4, 5, 6]]

Original: [[1, 2, 3], [4, 5, 6]]
Deep: [[99, 2, 3], [4, 5, 6]]
Original: [[87, 2, 3], [4, 5, 6]]
Deep: [[99, 2, 3], [4, 5, 6]]


### Lists and tuples

Lists and tuples are two fundamental data structures in Python, with lists being mutable ordered collections and tuples being immutable ordered collections.

List: created through "[]", we can change value it contains, slower for computation, suitable for dyanmic data

In [2]:
# Create a list
my_list = [1, 2, 3]

# Access elements
print(my_list[0])  # Output: 1

# Modify elements
my_list[0] = 10
print(my_list)  # Output: [10, 2, 3]

# Add elements
my_list.append(4)
print(my_list)  # Output: [10, 2, 3, 4]

# Remove elements
my_list.pop()
print(my_list)  # Output: [10, 2, 3]


1
[10, 2, 3]
[10, 2, 3, 4]
[10, 2, 3]


Tuple: created through "()", cannot change value after creation, fast computation, suitable for fixed data

In [1]:
# Create a tuple
my_tuple = (1, 2, 3)

# Access elements
print(my_tuple[0])  # Output: 1

# Cannot modify elements
# my_tuple[0] = 10  # Error: 'tuple' object does not support item assignment

# Tuples support slicing
print(my_tuple[1:])  # Output: (2, 3)

1
(2, 3)


- Use lists when data changes frequently.
- Use tuples when data is constant (e.g., coordinates, configuration settings).

### Class polymorphism

Class polymorphism provides a common interface that can be shared by subclasses (e.g. same method name, different)

First, create a base class providing a common interface with a method that can be overridden by subclasses.

In [3]:
class Shape:
    def area(self):
        raise NotImplementedError("Subclasses must override this method")

    def perimeter(self):
        raise NotImplementedError("Subclasses must override this method")

Then we define subclass implementing the area and perimeter methods differently, depending on the specific shape.

In [4]:
class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def area(self):
        return self.width * self.height

    def perimeter(self):
        return 2 * (self.width + self.height)

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius

    def area(self):
        return 3.14159 * self.radius ** 2

    def perimeter(self):
        return 2 * 3.14159 * self.radius

We can then use objects of Rectangle and Circle interchangeably when working with the Shape interface.

In [5]:
# List of shapes
shapes = [Rectangle(3, 4), Circle(5)]

# Polymorphic behavior
for shape in shapes:
    print(f"Area: {shape.area()}")
    print(f"Perimeter: {shape.perimeter()}")

Area: 12
Perimeter: 14
Area: 78.53975
Perimeter: 31.4159


This makes the code flexible and extensible for new shapes without modifying existing logic.

### Just-in-time compilation

- Just-In-Time (JIT) compilation is a technique to improve the performance of Python code by compiling parts of the code to machine code at runtime. 
- Python code is not normally converted to machine code at runtime. Instead, it is interpreted into bytecode stored in .pyc files, which is then executed by the Python interpreter.
- A common tool for JIT in Python is Numba.

In [6]:
# Import necessary modules
from numba import jit
import timeit

# Define functions
def sum_of_squares(n):
    total = 0
    for i in range(n):
        total += i * i
    return total

@jit
def sum_of_squares_jit(n):
    total = 0
    for i in range(n):
        total += i * i
    return total

# Input size
n = 10**6

# Time the functions
# Without JIT
%timeit -n 10 -r 3 sum_of_squares(n) #%timeit is a magic command that compute mean and std of execuation time of a function for r rounds, each round n times

# With JIT
# ensures the function is compiled before timing, so only runtime performance is measured
sum_of_squares_jit(n)

# Time JIT version
%timeit -n 10 -r 3 sum_of_squares_jit(n)

26.4 ms ± 109 μs per loop (mean ± std. dev. of 3 runs, 10 loops each)
103 ns ± 57 ns per loop (mean ± std. dev. of 3 runs, 10 loops each)
